In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

# Agrega las columnas de poblacion de interes
COLUMNAS_POBLACION_INTERES = {str(i): i for i in range(2017, 2036)}

## Obtencion de casos por area de influencia INT

In [3]:
# Lee los casos de area de influencia
casos_area_de_influencia = build_features.leer_casos_area_de_influencia(COLUMNAS_POBLACION_INTERES)

## Obtencion de porcentajes de Hemodinamia

In [11]:
# Define la ruta donde estan los porcentajes de Hemodinamia
RUTA_PLANILLA_PORCENTAJES = (
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_INT.xlsx"
)

# Solamente lee los diagnosticos y el porcentaje de Hemodinamia
datos_hmd = pd.read_excel(
    RUTA_PLANILLA_PORCENTAJES,
    sheet_name="porcentaje_trazadoras",
    usecols=["Diagnostico", "Porcentaje Pacientes Hemodinamia", "Horas en Hemodinamia"],
)

# Deja los diagnosticos que SI aportan a la cantidad de pabellones de HMD (porcentaje > 0)
datos_hmd = datos_hmd.query("`Porcentaje Pacientes Hemodinamia` > 0")

# Formatea la columna de diagnosticos
datos_hmd["Diagnostico"] = datos_hmd["Diagnostico"].str.split(" - ").str[0]

# Deja como indice a los diagnosticos
datos_hmd = datos_hmd.set_index("Diagnostico")

# Deja los diagnosticos como el indice
porcentaje_hmd = datos_hmd["Porcentaje Pacientes Hemodinamia"]

# Indica el tiempo quirurgico por diagnostico promedio (o 75%)
tiempos_hmd = datos_hmd["Horas en Hemodinamia"]

In [12]:
# Filtra los casos de area de influencia solamente a los diagnosticos que tengan un porcentaje de
# atencion cerrada mayor a 0
casos_area_de_influencia_para_hmd = casos_area_de_influencia[
    casos_area_de_influencia.index.isin(porcentaje_hmd.index)
]

In [14]:
# Se obtiene la cantidad de pacientes que van a hemodinamia
casos_hmd = casos_area_de_influencia_para_hmd[COLUMNAS_POBLACION_INTERES.values()].mul(
    porcentaje_hmd, axis=0
)

# Multiplica los casos en HMD con sus tiempos quirurgicos
tiempos_hmd = casos_hmd.mul(tiempos_hmd, axis=0)

# Obtiene la cantidad de horas laborales